In [1]:
import numpy as np
import pandas as pd
import os
import pybedtools
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
import seaborn as sns

In [3]:
%matplotlib inline

In [19]:
assembly_df=pd.read_csv("GCF_000001405.25_GRCh37.p13_assembly_report.txt",sep="\t",header=0)
assembly_df

,Sequence-Name,Sequence-Role,Assigned-Molecule,Assigned-Molecule-Location/Type,GenBank-Accn,Relationship,RefSeq-Accn,Assembly-Unit,Sequence-Length,UCSC-style-name
0,1,assembled-molecule,1,Chromosome,CM000663.1,=,NC_000001.10,Primary Assembly,249250621,chr1
1,2,assembled-molecule,2,Chromosome,CM000664.1,=,NC_000002.11,Primary Assembly,243199373,chr2
2,3,assembled-molecule,3,Chromosome,CM000665.1,=,NC_000003.11,Primary Assembly,198022430,chr3
3,4,assembled-molecule,4,Chromosome,CM000666.1,=,NC_000004.11,Primary Assembly,191154276,chr4
4,5,assembled-molecule,5,Chromosome,CM000667.1,=,NC_000005.9,Primary Assembly,180915260,chr5
5,6,assembled-molecule,6,Chromosome,CM000668.1,=,NC_000006.11,Primary Assembly,171115067,chr6
6,7,assembled-molecule,7,Chromosome,CM000669.1,=,NC_000007.13,Primary Assembly,159138663,chr7
7,8,assembled-molecule,8,Chromosome,CM000670.1,=,NC_000008.10,Primary Assembly,146364022,chr8
8,9,assembled-molecule,9,Chromosome,CM000671.1,=,NC_000009.11,Primary Assembly,141213431,chr9
9,10,assembled-molecule,10,Chromosome,CM000672.1,=,NC_000010.10,Primary Assembly,135534747,chr10


In [20]:
assembly_map={}
for index in assembly_df.index:
    key=assembly_df.loc[index]['RefSeq-Accn']
    value=assembly_df.loc[index]['UCSC-style-name']
    assembly_map[key]=value
assembly_map

{'NC_000001.10': 'chr1',
 'NC_000002.11': 'chr2',
 'NC_000003.11': 'chr3',
 'NC_000004.11': 'chr4',
 'NC_000005.9': 'chr5',
 'NC_000006.11': 'chr6',
 'NC_000007.13': 'chr7',
 'NC_000008.10': 'chr8',
 'NC_000009.11': 'chr9',
 'NC_000010.10': 'chr10',
 'NC_000011.9': 'chr11',
 'NC_000012.11': 'chr12',
 'NC_000013.10': 'chr13',
 'NC_000014.8': 'chr14',
 'NC_000015.9': 'chr15',
 'NC_000016.9': 'chr16',
 'NC_000017.10': 'chr17',
 'NC_000018.9': 'chr18',
 'NC_000019.9': 'chr19',
 'NC_000020.10': 'chr20',
 'NC_000021.8': 'chr21',
 'NC_000022.10': 'chr22',
 'NC_000023.10': 'chrX',
 'NC_000024.9': 'chrY',
 'NC_012920.1': 'chrM',
 'NT_113878.1': 'chr1_gl000191_random',
 'NT_113885.1': 'chr4_gl000193_random',
 'NT_113888.1': 'chr4_gl000194_random',
 'NT_113889.1': 'chrUn_gl000218',
 'NT_113891.2': 'chr6_cox_hap2',
 'NT_113901.1': 'chr7_gl000195_random',
 'NT_113907.1': 'chr8_gl000197_random',
 'NT_113909.1': 'chr8_gl000196_random',
 'NT_113911.1': 'chr9_gl000201_random',
 'NT_113914.1': 'chr9_gl0

In [22]:
outputfile_new=open("NEW_GCA_000001405.25_GRCh37_Refgene.txt","w")
with open("GCA_000001405.25_GRCh37_Refgene.txt","r") as inputfile:
    for inputline in inputfile:
        inputline=inputline.strip()
        if inputline!="":
            inputitems=inputline.strip().split("\t")
            key=inputitems[0]
            value=assembly_map[key]
            inputitems[0]=value
            newline="\t".join(inputitems)
            outputfile_new.write(newline+"\n")
outputfile_new.close()

In [25]:
#split by + and - strand
inputfilename="NEW_GCA_000001405.25_GRCh37_Refgene.txt"
outputfilename_F="GCA_000001405.25_GRCh37_Refgene_F.bed"
outputfile_F=open(outputfilename_F,"w")
outputfilename_R="GCA_000001405.25_GRCh37_Refgene_R.bed"
outputfile_R=open(outputfilename_R,"w")
with open(inputfilename,"r") as inputfile:
    for inputline in inputfile:
        inputline=inputline.strip()
        if inputline!="":
            inputitems=inputline.split("\t")
            chromnum=inputitems[0].strip()
            startpos=inputitems[3].strip()
            stoppos=inputitems[4].strip()
            strand=inputitems[6].strip()
            metainfo_items=inputitems[8].strip().split(";")
            genename=""
            for infoitem in metainfo_items:
                if len(infoitem)>5 and infoitem[0:5]=="gene=":
                    genename=infoitem.replace("gene=","")
            if genename!="":
                outputstring=chromnum+"\t"+startpos+"\t"+stoppos+"\t"+genename
                if strand=="+":
                    outputfile_F.write(outputstring+"\n")
                elif strand=="-":
                    outputfile_R.write(outputstring+"\n")

outputfile_F.close()
outputfile_R.close()

In [26]:
a=pybedtools.BedTool("GCA_000001405.25_GRCh37_Refgene_F.bed")
a=a.sort()
a.moveto("GCA_000001405.25_GRCh37_Refgene_F_sorted.bed")

a=pybedtools.BedTool("GCA_000001405.25_GRCh37_Refgene_R.bed")
a=a.sort()
a.moveto("GCA_000001405.25_GRCh37_Refgene_R_sorted.bed")

<BedTool(GCA_000001405.25_GRCh37_Refgene_R_sorted.bed)>

In [7]:
#------Production line------

def generate_list_from(sorted_onebasepair_bedgraph):
    total_list=[]
    inputfilename=sorted_onebasepair_bedgraph
    with open(inputfilename,"r") as inputfile:
        for inputline in inputfile:
            inputline=inputline.strip()
            if inputline!="":
                inputitems=inputline.split("\t")
                if len(inputitems)==4:
                    chromnum=inputitems[0]
                    startpos=int(inputitems[1])
                    stoppos=int(inputitems[2])
                    countvalue=int(inputitems[3])
                    vector=(chromnum,startpos,stoppos,countvalue)
                    total_list.append(vector)
    return total_list
def preprocess_bedgraph(rawbedgraph, sorted_onebasepair_bedgraph):
    tempbed=rawbedgraph+".temp"
    tailcommand="tail -n +3 "+rawbedgraph+" > "+tempbed
    os.system(tailcommand)
    sortcommand="sort -k1,1 -k2,2n "+tempbed+" > "+sorted_onebasepair_bedgraph
    os.system(sortcommand)
    os.system("rm "+tempbed)
    
def ifsmaller(chr1,num1,chr2,num2):
    result=False
    if chr1==chr2:
        if num1<num2:
            result=True
    elif chr1<chr2:
        result=True
    return result

def iflarger(chr1,num1,chr2,num2):
    result=False
    if chr1==chr2:
        if num1>num2:
            result=True
    elif chr1>chr2:
        result=True
    return result

def sum_of_values(snpfilename, bedfilename):
    #snpfilename="AAVS1_32hr_F_1.bed"
    #bedfilename="GCA_000001405.15_GRCh38_Refgene_F.bed"

    outputfilename="geneproseqcount_"+snpfilename
    outputfilename=outputfilename.replace(".bed",".txt")



    onebase_bedfile_list=generate_list_from(snpfilename)
    outputfile=open(outputfilename,"w")

    list_start_pos=0
    list_stop_pos=0

    current_value_list=[]

    bedlinenum=0
    bedfile=open(bedfilename,'r')
    bedline=bedfile.readline()
    bedlinenum=bedlinenum+1
    while bedline:
        bedline=bedline.strip()
        if bedline!="":
            bedlineitems=bedline.split("\t")
            bedlinechr=bedlineitems[0].strip()
            bedstartpos=int(bedlineitems[1].strip())
            bedendpos=int(bedlineitems[2].strip())
            genename=bedlineitems[3].strip()
            current_value_list=[]
            while list_start_pos<len(onebase_bedfile_list):
                snpfileitems=onebase_bedfile_list[list_start_pos]
                if len(snpfileitems)>0:
                    snpchr1=snpfileitems[0]
                    snppos1=int(snpfileitems[1])
                    if ifsmaller(snpchr1,snppos1,bedlinechr,bedstartpos)==True:
                        list_start_pos=list_start_pos+1
                    else:
                        break
            list_stop_pos=list_start_pos  
            while list_stop_pos<len(onebase_bedfile_list):
                snpfileitems=onebase_bedfile_list[list_stop_pos]
                if len(snpfileitems)>0:
                    snpchr2=snpfileitems[0]
                    snppos2=int(snpfileitems[1])
                    if iflarger(snpchr2,snppos2,bedlinechr,bedendpos)==False:
                        list_stop_pos=list_stop_pos+1
                        current_value_list.append(snpfileitems[3])
                    else:
                        break
            gapnum=np.sum(current_value_list)
            density=float(gapnum)/float(bedendpos-bedstartpos)
            outputstring=bedline+"\t"+str(gapnum)
            outputfile.write(outputstring+"\n")

        bedline=bedfile.readline()
        bedlinenum=bedlinenum+1
    bedfile.close()

    outputfile.close()
    
def run_count_sum_from_bedgraph():
    
    # These bed graphs need to be downloaded from 
    # https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE150530
    rawdata_folder="/data/"
    rawbedgraph_list=[
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR56_AAVS1-32hr-rep1_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR56_AAVS1-32hr-rep1_R.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR57_AAVS1-32hr-rep2_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR57_AAVS1-32hr-rep2_R.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR58_BCLKO-32hr-rep1_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR58_BCLKO-32hr-rep1_R.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR59_BCLKO-32hr-rep2_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR59_BCLKO-32hr-rep2_R.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR60_AAVS1-72hr-rep1_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR60_AAVS1-72hr-rep1_R.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR61_AAVS1-72hr-rep2_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR61_AAVS1-72hr-rep2_R.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR62_BCLKO-72hr-rep1_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR62_BCLKO-72hr-rep1_R.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR63_BCLKO-72hr-rep2_F.bedGraph",
        "hg19_primaryHSPC_CD34+_PROseq_Orkin001merged_PR63_BCLKO-72hr-rep2_R.bedGraph"
    ]
    sorted_onebasepair_bedgraph_list=[
        "AAVS1_32hr_F_1.bed",
        "AAVS1_32hr_R_1.bed",
        "AAVS1_32hr_F_2.bed",
        "AAVS1_32hr_R_2.bed",
        "BCL11AKO_32hr_F_1.bed",
        "BCL11AKO_32hr_R_1.bed",
        "BCL11AKO_32hr_F_2.bed",
        "BCL11AKO_32hr_R_2.bed",
        "AAVS1_72hr_F_1.bed",
        "AAVS1_72hr_R_1.bed",
        "AAVS1_72hr_F_2.bed",
        "AAVS1_72hr_R_2.bed",
        "BCL11AKO_72hr_F_1.bed",
        "BCL11AKO_72hr_R_1.bed",
        "BCL11AKO_72hr_F_2.bed",
        "BCL11AKO_72hr_R_2.bed"
    ]
    bedfilename_list=[
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_F_sorted.bed",
        "GCA_000001405.25_GRCh37_Refgene_R_sorted.bed"
    ]
    
    for i in range(0,len(bedfilename_list)):
        rawbedgraph=os.path.join(rawdata_folder,rawbedgraph_list[i])
        sorted_onebasepair_bedgraph=sorted_onebasepair_bedgraph_list[i]
        bedfilename=bedfilename_list[i]
        print(rawbedgraph)
        preprocess_bedgraph(rawbedgraph,sorted_onebasepair_bedgraph)
        sum_of_values(sorted_onebasepair_bedgraph, bedfilename)

run_count_sum_from_bedgraph()        

In [32]:
#combine results

a=pd.read_csv("geneproseqcount_AAVS1_32hr_F_1.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_AAVS1_32hr_R_1.txt",sep="\t",header=None)
frames=[a,b]
result_AAVS1_32hr_1 = pd.concat(frames,ignore_index=True)

a=pd.read_csv("geneproseqcount_AAVS1_32hr_F_2.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_AAVS1_32hr_R_2.txt",sep="\t",header=None)
frames=[a,b]
result_AAVS1_32hr_2 = pd.concat(frames,ignore_index=True)

a=pd.read_csv("geneproseqcount_BCL11AKO_32hr_F_1.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_BCL11AKO_32hr_R_1.txt",sep="\t",header=None)
frames=[a,b]
result_BCL11AKO_32hr_1 = pd.concat(frames,ignore_index=True)

a=pd.read_csv("geneproseqcount_BCL11AKO_32hr_F_2.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_BCL11AKO_32hr_R_2.txt",sep="\t",header=None)
frames=[a,b]
result_BCL11AKO_32hr_2 = pd.concat(frames,ignore_index=True)

a=pd.read_csv("geneproseqcount_AAVS1_72hr_F_1.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_AAVS1_72hr_R_1.txt",sep="\t",header=None)
frames=[a,b]
result_AAVS1_72hr_1 = pd.concat(frames,ignore_index=True)

a=pd.read_csv("geneproseqcount_AAVS1_72hr_F_2.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_AAVS1_72hr_R_2.txt",sep="\t",header=None)
frames=[a,b]
result_AAVS1_72hr_2 = pd.concat(frames,ignore_index=True)

a=pd.read_csv("geneproseqcount_BCL11AKO_72hr_F_1.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_BCL11AKO_72hr_R_1.txt",sep="\t",header=None)
frames=[a,b]
result_BCL11AKO_72hr_1 = pd.concat(frames,ignore_index=True)

a=pd.read_csv("geneproseqcount_BCL11AKO_72hr_F_2.txt",sep="\t",header=None)
b=pd.read_csv("geneproseqcount_BCL11AKO_72hr_R_2.txt",sep="\t",header=None)
frames=[a,b]
result_BCL11AKO_72hr_2 = pd.concat(frames,ignore_index=True)

In [33]:
sum_of_reads={
    'BCL11AKO_32hr_1': 34480558,
    'BCL11AKO_32hr_2': 41372273,
    'AAVS1_32hr_2': 42432538,
    'AAVS1_32hr_1': 51715016,
    'BCL11AKO_72hr_1': 47811204,
    'BCL11AKO_72hr_2': 44465612,
    'AAVS1_72hr_1': 47971998,
    'AAVS1_72hr_2': 45951100
}

In [34]:
columns=['AAVS1_32hr_1','AAVS1_32hr_2','BCL11AKO_32hr_1','BCL11AKO_32hr_2',
        'AAVS1_72hr_1','AAVS1_72hr_2','BCL11AKO_72hr_1','BCL11AKO_72hr_2']
result_combined=result_AAVS1_32hr_1.copy()
result_combined.columns=['chrom','start','end','genename','AAVS1_32hr_1']

for column_name in columns[1:]:
    print(column_name)
    dataframe_name='result_'+column_name
    value_list=globals().get(dataframe_name,None)[4].tolist()
    result_combined[column_name]=value_list

AAVS1_32hr_2
BCL11AKO_32hr_1
BCL11AKO_32hr_2
AAVS1_72hr_1
AAVS1_72hr_2
BCL11AKO_72hr_1
BCL11AKO_72hr_2


In [35]:
result_combined.to_csv("geneproseqcount_combined.txt",sep="\t",header=True,index=None)

In [36]:
result_combined_norm=pd.read_csv("geneproseqcount_combined.txt",sep="\t",header=0)
for column_name in columns:
    value_list=result_combined_norm[column_name]
    value_normalized=np.array(value_list)/sum_of_reads[column_name]*1000000
    new_column_name="norm_"+column_name
    result_combined_norm[new_column_name]=list(value_normalized)
#result_combined_norm.to_csv("geneproseqcount_combined_norm.csv",sep=",",header=True,index=None)

In [37]:
L2FC_32hr_list=[]
L2FC_72hr_list=[]
for index in result_combined_norm.index:
    bcl11ako_32=(1E-10)+(result_combined_norm.loc[index]['norm_BCL11AKO_32hr_1']+result_combined_norm.loc[index]['norm_BCL11AKO_32hr_2'])
    control_32=(1E-10)+(result_combined_norm.loc[index]['norm_AAVS1_32hr_1']+result_combined_norm.loc[index]['norm_AAVS1_32hr_2'])
    l2fc_32=np.log2(bcl11ako_32/control_32)
    L2FC_32hr_list.append(l2fc_32)
    bcl11ako_72=(1E-10)+(result_combined_norm.loc[index]['norm_BCL11AKO_72hr_1']+result_combined_norm.loc[index]['norm_BCL11AKO_72hr_2'])
    control_72=(1E-10)+(result_combined_norm.loc[index]['norm_AAVS1_72hr_1']+result_combined_norm.loc[index]['norm_AAVS1_72hr_2'])
    l2fc_72=np.log2(bcl11ako_72/control_72)
    L2FC_72hr_list.append(l2fc_72)

result_combined_norm['L2FC_32hr']=L2FC_32hr_list
result_combined_norm['L2FC_72hr']=L2FC_72hr_list

result_combined_norm.to_csv("geneproseqcount_combined_norm.csv",sep=",",header=True,index=None)

In [38]:
result_combined_norm.head()

,chrom,start,end,genename,AAVS1_32hr_1,AAVS1_32hr_2,BCL11AKO_32hr_1,BCL11AKO_32hr_2,AAVS1_72hr_1,AAVS1_72hr_2,...,norm_AAVS1_32hr_1,norm_AAVS1_32hr_2,norm_BCL11AKO_32hr_1,norm_BCL11AKO_32hr_2,norm_AAVS1_72hr_1,norm_AAVS1_72hr_2,norm_BCL11AKO_72hr_1,norm_BCL11AKO_72hr_2,L2FC_32hr,L2FC_72hr
0,chr1,30366,30503,MIR1302-2,5.0,4.0,3.0,4.0,4.0,7.0,...,0.096684,0.094267,0.087006,0.096683,0.083382,0.152336,0.209156,0.067468,-0.055940,0.230865
1,chr1,69091,70008,OR4F5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.019337,0.000000,0.000000,0.000000,0.000000,0.021762,0.020916,0.022489,-27.526770,0.996029
2,chr1,323892,328581,LOC100132287,25.0,11.0,20.0,26.0,36.0,44.0,...,0.483419,0.259235,0.580037,0.628440,0.750438,0.957540,0.627468,0.607211,0.702429,-0.468153
3,chr1,367659,368597,OR4F29,3.0,1.0,2.0,2.0,0.0,5.0,...,0.058010,0.023567,0.058004,0.048342,0.000000,0.108811,0.062747,0.044979,0.382520,-0.014471
4,chr1,752751,755214,FAM87B,0.0,0.0,1.0,1.0,0.0,0.0,...,0.000000,0.000000,0.029002,0.024171,0.000000,0.000000,0.020916,0.000000,28.986109,27.640004
